In [1]:
import pandas as pd
import json
import gzip
import ast

In [2]:
review = "../Dataset/user_reviews.json.gz"
data = []

with gzip.open(review, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

reviews = pd.DataFrame(data)

In [3]:
reviews.head(10)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
5,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"[{'funny': '1 person found this review funny',..."
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l..."
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,"[{'funny': '5 people found this review funny',..."
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."


Desanidamos la columna reviews

In [4]:
reviews_open = reviews.explode('reviews')
reviews_open = pd.concat([reviews_open.drop(['reviews'], axis=1), reviews_open['reviews'].apply(pd.Series)], axis=1)

In [8]:
reviews_open.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      59333 non-null  object 
 1   user_url     59333 non-null  object 
 2   funny        59305 non-null  object 
 3   posted       59305 non-null  object 
 4   last_edited  59305 non-null  object 
 5   item_id      59305 non-null  object 
 6   helpful      59305 non-null  object 
 7   recommend    59305 non-null  object 
 8   review       59305 non-null  object 
 9   0            0 non-null      float64
 10  posted year  49186 non-null  object 
dtypes: float64(1), object(10)
memory usage: 5.4+ MB


Creo la columna year

In [7]:
# Extraigo el año de la columna 'posted' y lo guardo en una nueva columna 'posted year'
# extract(r'(\d{4})') es una expresión regular para extraer una secuencia de 4 dígitos que en este caso seria el año
reviews_open['posted year'] = reviews_open['posted'].str.extract(r'(\d{4})') 

Elimino la columna posted

In [9]:
reviews_open.drop('posted', axis=1, inplace=True)

In [10]:
reviews_open.head(2)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011


Deberemos hacer el analisis de sentimientos 